In [5]:
# Measured with a meter stick, unceratinly 0.5 mm
solenoidLength = 389 # mm

# Measured with a vernier calipers, uncertainty 0.05 mm
# Note: External Diameter
solenoidDiameter = 44 # mm

In [6]:
# Average taken accross 5 measurements: 10,10,11,11,12
copperTurns = 1212

In [ ]:
frequency = [625000, 28000, 6500]
voltage = [63.45, 24.72, 6.81]